In [ ]:
# !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"

In [4]:
from dqn_atari import Args, make_env, QNetwork
import gymnasium as gym
import torch
import json
import numpy as np

In [2]:
args = Args
args.env_id = 'PongNoFrameskip-v4'

In [5]:
envs = gym.vector.SyncVectorEnv(
        [make_env(args.env_id, args.seed + i, i, args.capture_video, 'xsr') for i in range(args.num_envs)]
    )

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [6]:
q_network = QNetwork(envs)
q_network.load_state_dict(torch.load('dqn_atari.cleanrl_model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [11]:
states = np.load('data.npz')['observation']
states.shape

(10000, 4, 84, 84)

In [27]:
with open('submit_0.json') as f:
    example = json.load(f)
example[:2]

[{'advantages': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'value': 0.0,
  'qvalues': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]},
 {'advantages': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'value': 0.0,
  'qvalues': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}]

In [25]:
ans = []
for state in states:
    obs = torch.from_numpy(state).unsqueeze(0).float()
    q_values = q_network(obs)
    actions = torch.argmax(q_values, dim=1)
    values = q_values.gather(1, actions.unsqueeze(1))
    advantages = (q_values - values).squeeze().cpu().detach().numpy().tolist()      # advantages: A(s, a) = Q(s, a) - V(s)
    ans.append({'advantages': advantages,
                'value': values.item(),
                'qvalues': q_values.tolist()[0]})

In [29]:
with open('submit.json', 'w') as f:
    json.dump(ans, f)